In [1]:
import pandas as pd
import numpy as np
import os

In [4]:

        
# Leitura inicial sem parse_dates e sem index_col
dfs = pd.read_csv("dados/limpos/ABEV3.SA_20250330_limpo.csv",skiprows=[1,2] ,header=0, dtype=str, delimiter=',')
#dfs = calcular_indicadores(dfs)
#print(dfs.describe())
#print(dfs.head())
colunas_para_float = ["PrecoFechamento", "PrecoMaximo", "PrecoMinimo", "PrecoAbertura", "VolumeNegociado"]
dfs["Date"] = pd.to_datetime(dfs["Date"], errors="coerce")
for col in colunas_para_float:
    dfs[col] = pd.to_numeric(dfs[col], errors="coerce")
print(dfs.dtypes)     
      
        

Date               datetime64[ns]
PrecoFechamento           float64
PrecoMaximo               float64
PrecoMinimo               float64
PrecoAbertura             float64
VolumeNegociado             int64
dtype: object


In [3]:
def ajustarvar(dfs):
    colunas_para_float = ["PrecoFechamento", "PrecoMaximo", "PrecoMinimo", "PrecoAbertura", "VolumeNegociado"]
    dfs["Date"] = pd.to_datetime(dfs["Date"], errors="coerce")
    for col in colunas_para_float:
        dfs[col] = pd.to_numeric(dfs[col], errors="coerce")
    print("conversão para valores numericos...")
    return dfs

def calcular_indicadores(df):
    # Exemplo simples: Retorno percentual diário
    df["DifPreco"] = df["PrecoFechamento"].diff()
    df["Retorno_pct"] = df["PrecoFechamento"].pct_change()
    
    # ganho e perda
    df["Ganho"] = df["DifPreco"].clip(lower=0)
    df["Perda"] = -1 * df["DifPreco"].clip(upper=0)
    df["MediaGanho"] = df["Ganho"].rolling(15).mean()
    df["MediaPerda"] = df["Perda"].rolling(15).mean()
    
    # Exemplo: média móvel de 20 dias
    df["MediaMovel_5"] = df["PrecoFechamento"].rolling(5).mean()
    df["MediaMovel_15"] = df["PrecoFechamento"].rolling(15).mean()
    df["MediaMovel_30"] = df["PrecoFechamento"].rolling(30).mean()
    df["MediaMovel_60"] = df["PrecoFechamento"].rolling(60).mean()
    df["MediaMovel_90"] = df["PrecoFechamento"].rolling(90).mean()

    
    
    print(df.head())
    return df


def main():
    caminho_limpos = "dados/limpos"
    
    arquivos_csv = [
        os.path.join(caminho_limpos, f)
        for f in os.listdir(caminho_limpos)
        if f.endswith(".csv")
    ]
    
    os.makedirs("dados/indicadores", exist_ok=True)

    for arquivo_csv in arquivos_csv:
        print(f"Processando arquivo: {arquivo_csv}")
        
        # Leitura inicial sem parse_dates e sem index_col
        dfs = pd.read_csv(arquivo_csv,skiprows=[1,2] ,header=0, dtype=str, delimiter=',')
        
        dfs = ajustarvar(dfs)
        
        dfs = calcular_indicadores(dfs)
    
       # Gera nome do arquivo de saída baseado no arquivo de entrada
        nome_arquivo = os.path.basename(arquivo_csv).replace("_limpo.csv", "_indicadores.csv")
        caminho_indicadores = os.path.join("dados", "indicadores", nome_arquivo)

        # Salva DataFrame final
        dfs.to_csv(caminho_indicadores, index=False, sep=',', encoding='utf-8')
        print(f"Arquivo de indicadores salvo em: {caminho_indicadores}")
        


if __name__ == "__main__":
    main()

Processando arquivo: dados/limpos\ABEV3.SA_20250330_limpo.csv
conversão para valores numericos...
        Date  PrecoFechamento  PrecoMaximo  PrecoMinimo  PrecoAbertura  \
0 2024-01-04        13.263288    13.292438    13.136971      13.214705   
1 2024-01-05        13.214705    13.302154    13.146687      13.234137   
2 2024-01-08        13.321589    13.360455    13.195271      13.224421   
3 2024-01-09        13.234138    13.302154    13.166121      13.302154   
4 2024-01-10        13.370173    13.438189    13.214706      13.234138   

   VolumeNegociado  DifPreco  Retorno_pct     Ganho     Perda  MediaGanho  \
0         19996000       NaN          NaN       NaN       NaN         NaN   
1         18293900 -0.048583    -0.003663  0.000000  0.048583         NaN   
2         11014600  0.106884     0.008088  0.106884 -0.000000         NaN   
3         15705100 -0.087451    -0.006565  0.000000  0.087451         NaN   
4         18313300  0.136035     0.010279  0.136035 -0.000000         Na